# Import the Module and authenticate google earth engine

In [ ]:
import geemap, ee

In [8]:
ee.Initialize()

# Data import and conversion to earth engine object

In [11]:
study_area_shp = 'shapefile/bardiya.shp'
aoi = geemap.shp_to_ee(study_area_shp)

# Calculation of vegetation indices GEE platforms

## EVI

In [12]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8').divide(10000),
            'RED': image.select('B4').divide(10000),
            'BLUE': image.select('B2').divide(10000)
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

### GLI (Green Leaf Index)

In [13]:
# // GLI (Green Leaf Index)
def getGLI(image):
    # Compute the EVI using an expression.
    GLI = image.expression ('(((GREEN - RED) + (GREEN - BLUE)) / ((2 * GREEN) + RED + BLUE))', {
        'GREEN': image.select ('B3'),  
        'RED': image.select ('B4'),
        'BLUE': image.select ('B2')
        }).rename("GLI")
    image = image.addBands(EVI)

    return(image)


### SAVI (Soil Adjusted Vegetation Index)

In [14]:
# SAVI (Soil Adjusted Vegetation Index)
def getSAVI(image):
    # Compute the EVI using an expression.
    SAVI = image.expression ('(((NIR - RED) / (NIR + RED + L))* (1+L))',{
        'L': 0.5, # Cobertura vegetation 0-1
        'NIR': image.select ('B8'),
        'RED': image.select ('B4')
    }).rename("SAVI")
    
    image = image.addBands(SAVI)

    return(image)

### GCI (Green Chlorophyll Index)

In [15]:
# GCI (Green Chlorophyll Index)
def getGCI(image):
    # Compute the EVI using an expression.
    GCI = image.expression ('(((NIR) / (GREEN)) - 1)', {
        'NIR': image.select ('B8'),  
        'GREEN': image.select ('B3')
        }).rename("GCI")
    
    image = image.addBands(GCI)

    return(image)

### RGR (Red Green Ratio)

In [16]:
# RGR (Red Green Ratio)
def getRGR(image):
    # Compute the EVI using an expression.
    RGR = image.expression ('((RED) / (GREEN))', {
        'RED': image.select ('B4'),  
        'GREEN': image.select ('B3')
    }).rename("RGR")
    
    image = image.addBands(RGR)

    return(image)

### SIPI (Structure Insensitive Pigment Index)

In [17]:
# SIPI (Structure Insensitive Pigment Index)
def getSIPI(image):
    # Compute the EVI using an expression.
    SIPI = image.expression ('((NIR - BLUE) / (NIR - RED))',{
        'NIR': image.select ('B8'),
        'BLUE': image.select ('B2'), 
        'RED': image.select ('B4')
    }).rename("SIPI")
    
    image = image.addBands(SIPI)

    return(image)

### ARVI (Atmospherically Resistant Vegetation Index)

In [18]:
# ARVI (Atmospherically Resistant Vegetation Index)
def getARVI(image):
    # Compute the EVI using an expression.
    ARVI = image.expression ('((NIR - (2 * RED) + BLUE) / (NIR + (2 * RED) + BLUE))',{
        'NIR': image.select ('B8'),
        'BLUE': image.select ('B2'), 
        'RED': image.select ('B4')
    }).rename("ARVI")
    
    image = image.addBands(ARVI)

    return(image)

### NBRI (Normalized Burned Ratio Index)

In [22]:
# NBRI (Normalized Burned Ratio Index)
def getNBRI(image):
    # Compute the EVI using an expression.
    NBRI = image.expression ('(NIR - SWIR) / (NIR + SWIR)', {
        'NIR': image.select ('B8'),  
        'SWIR': image.select ('B12')
    }).rename("NBRI")
    
    image = image.addBands(NBRI)

    return(image)

# filter image for your desired dates and apply vegetation indices calculation

In [23]:
Sentinel_data = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate("2022-03-01","2022-03-31").filterBounds(aoi) \
    .map(getNBRI).median()

## Visualization parameters 

In [24]:
color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":1, 'palette':color}

# Add layers and visualize on Map

In [25]:
# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 8)
map1.addLayer(Sentinel_data.clip(aoi).select('NBRI'), pallete, "NBRI")

map1.addLayerControl()
map1

Map(center=[28.741569646024022, 80.87937460930668], controls=(WidgetControl(options=['position', 'transparent_…